In [66]:
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [67]:
replacewords = ['-', '.','&']

In [68]:
def address_similarity(address1, address2):
    stopwords = ['of', 'and', ',', 'national association', 'na', 'corporation', 'company', 'n.a.', 'inc.', 'inc', 'ssb', 'the']
    
    address1_tokens = [word.lower() for word in address1.split() if word.lower() not in stopwords]
    address2_tokens = [word.lower() for word in address2.split() if word.lower() not in stopwords]
    return fuzz.ratio(' '.join(address1_tokens), ' '.join(address2_tokens))
   # return fuzz.ratio(address1, address2)

# def address_similarity(address1, address2):
#     stopwords = ['of','and',',','national association','na', '&','-','.','corporation','company','n.a.','inc.','inc','ssb','the']
#     # stop_words = set(stopwords.words('english'))
#     address1_tokens = [word.lower() for word in address1.split() if word.lower() not in stopwords]
#     address2_tokens = [word.lower() for word in address2.split() if word.lower() not in stopwords]
# 
#     return fuzz.ratio(' '.join(address1_tokens), ' '.join(address2_tokens))

In [69]:
data_ai_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Projects/Dewey/data/data_ai/data_ai_data_cleaned.csv" 
data_ai = pd.read_csv(data_ai_file)

data_ai['Company_Name'] = data_ai['Company_Name'].str.lower()

for sw in replacewords:
    data_ai['Company_Name'] = data_ai['Company_Name'].str.replace(sw, ' ')

data_ai['Company_Name'] = data_ai['Company_Name'].str.replace('  ', ' ')

In [70]:
sod_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/SOD/data/ALL_2022.csv" 
sod = pd.read_csv(sod_file, encoding='latin1')
sod = sod.drop_duplicates(subset="NAMEFULL")
sod = sod[['NAMEFULL', 'RSSDID']]
sod = sod.rename(columns={'NAMEFULL': 'bank_name'})

bank_names_file ="C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Projects/Dewey/data/bank_names.csv"
bank_names = pd.read_csv(bank_names_file, encoding='latin1')
sod = pd.concat([sod, bank_names], axis=0)

sod['bank_name'] = sod['bank_name'].str.lower()

for sw in replacewords:
    sod['bank_name'] = sod['bank_name'].str.replace(sw, ' ')

sod['bank_name'] = sod['bank_name'].str.replace('  ', ' ')

In [71]:
output_file_path = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/data_ai_match.csv"  

with tqdm(total=len(sod) ) as pbar:
    with open(output_file_path, "a") as output_file:
        for index1, row1 in sod.iterrows():
            for index2, row2 in data_ai.iterrows():
                try:
                        similarity_score = address_similarity(row1['bank_name'], row2['Company_Name'])
                        if similarity_score >= 75:# & row1['bank_name'][:3].lower() == row2['Company_Name'][:3].lower(): 
                            output_file.write(f"{row1['bank_name']}|{row1['RSSDID']}|{row2['Company_Name']}|{row2['App']}|{row2['App_Name']}|{similarity_score}\n")
                except:
                    pass
            pbar.update(1)

100%|██████████| 11033/11033 [35:58<00:00,  5.11it/s]


In [33]:
print((f"{row1['bank_name']}|{row1['RSSDID']}|{row2['Company_Name']}|{row2['App']}|{row2['App_Name']}|{similarity_score}\n"))

BERKSHIRE BANK MUNICIPAL BANK|3384466|Northfield Bank|533181082|Northfield Bank – Mobile Bank|34
